In [1]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from keras.applications import NASNetMobile
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = '3'


2023-08-11 13:39:57.605602: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 13:39:58.411166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Train_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['file_path'].to_list()
Train_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['standard'].to_list()
Test_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['file_path'].to_list()
Test_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['standard'].to_list()
Val_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['file_path'].to_list()
Val_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['standard'].to_list()

In [3]:
Train_img_path='../../data/standardFrame_data/scale_skip/train'
Test_img_path='../../data/standardFrame_data/scale_skip/test'
Val_img_path='../../data/standardFrame_data/scale_skip/val'

In [4]:
size=224
x_train = np.zeros((len(Train_img_list),size,size,3))
for i in range(len(Train_img_list)):
    x_train[i] =np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)))
x_train=x_train/255
y_train=np.array(Train_label_list)

x_test = np.zeros((len(Test_img_list),size,size,3))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open(Test_img_path+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)

x_val = np.zeros((len(Val_img_list),size,size,3))
for i in range(len(Val_img_list)):
    x_val[i] =np.array(Image.open(Val_img_path+Val_img_list[i]).resize((size,size)))
x_val=x_val/255
y_val=np.array(Val_label_list)

In [5]:
def batch_generator(image, label, batchsize):
    N = len(image)
    indices = np.arange(N)  # 0부터 N-1까지의 인덱스 배열 생성
    np.random.shuffle(indices)  # 인덱스 배열을 무작위로 섞음

    i = 0
    while True:
        if i + batchsize <= N:
            batch_indices = indices[i:i+batchsize]
            i = i + batchsize
        else:  # 남은 데이터가 batchsize보다 작을 때, 배열을 wrap around하여 다시 섞음
            batch_indices = np.concatenate((indices[i:], indices[:batchsize - (N - i)]))
            i = batchsize - (N - i)

            np.random.shuffle(indices)  # 다음 에포크를 위해 인덱스 배열을 무작위로 섞음

        yield image[batch_indices], label[batch_indices]
        
checkpoint_filepath = "../../model/skip/NASNetMobile_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}

input_t=K.Input(shape=(size,size, 3))
input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
ResNet=NASNetMobile(include_top=True,weights='imagenet',input_tensor=input_tensor)
model = K.models.Sequential()
model.add(ResNet)
model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(K.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=K.optimizers.Adam(lr=2e-4),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
histo=model.fit(
    batch_generator(x_train,y_train,64),
    validation_data=(x_val,y_val),
    epochs=500,
    steps_per_epoch=len(x_train)//64,
    callbacks=model_checkpoint_callback,
    shuffle=True,
    class_weight=class_weight
)
model.save('../../model/skip/NASNetMobile.h5')

2023-08-11 13:43:41.651871: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38163 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:61:00.0, compute capability: 8.0


Epoch 1/500


2023-08-11 13:43:47.921558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-11 13:44:30.183189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-11 13:44:31.214734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-11 13:44:31.218698: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55ad8b141b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-11 13:44:31.218733: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2023-08-11 13:44:31.242985:

388/388 [==============================] - 175s 193ms/step - loss: 0.5323 - accuracy: 0.7061 - val_loss: 0.2715 - val_accuracy: 0.9335
Epoch 2/500
388/388 [==============================] - 68s 174ms/step - loss: 0.2751 - accuracy: 0.8812 - val_loss: 0.3404 - val_accuracy: 0.9335
Epoch 3/500
388/388 [==============================] - 65s 168ms/step - loss: 0.1871 - accuracy: 0.9187 - val_loss: 1.4775 - val_accuracy: 0.6756
Epoch 4/500
388/388 [==============================] - 65s 166ms/step - loss: 0.1584 - accuracy: 0.9335 - val_loss: 0.4571 - val_accuracy: 0.9127
Epoch 5/500
388/388 [==============================] - 66s 169ms/step - loss: 0.1234 - accuracy: 0.9480 - val_loss: 1.7984 - val_accuracy: 0.6613
Epoch 6/500
388/388 [==============================] - 67s 173ms/step - loss: 0.1057 - accuracy: 0.9575 - val_loss: 0.2620 - val_accuracy: 0.9335
Epoch 7/500
388/388 [==============================] - 65s 167ms/step - loss: 0.1063 - accuracy: 0.9571 - val_loss: 0.8012 - val_accura